In [1]:
import pandas as pd
import keras
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = 9,6

from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import random
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings(action='ignore')

from generate_models import generate_model1, generate_model2, generate_model3

# Data loading and preparation

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
BATCH_SIZE = 32
INPUT_SHAPE = X_train.shape[1:]
NUM_CLASSES=10

print('x_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [4]:
# Normalize input data
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

# Convert class labels to one-hot encoded
y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

# Fitting the custom deep learning models

In [5]:
LR=1e-3 # chosen arbitrary, tuning performed later on for further experiments
EPOCHS=100  
PATIENCE=10
MIN_DELTA=0.01
early_stop = EarlyStopping(monitor='val_accuracy', patience=PATIENCE, min_delta=MIN_DELTA)

In [6]:
model1 = generate_model1(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
model2 = generate_model2(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
model3=generate_model3(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

In [7]:
model1.summary()
model2.summary()
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 16)        448       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                        

# Fitting pre-trained arch